In [3]:
import pandas as pd
from clingo import Control
import ast
from matching_utils import (
    get_invalid_matches, 
    check_mentor_capacity,
    check_day_conflicts,
    check_multiple_days
)

students_df =pd.read_csv('../DATASETS/studenten.csv')
mentors_df =pd.read_csv('../DATASETS/mentoren.csv')

education_mapping = {
    'Associate': 1,
    'Bachelor': 2, 
    'Master': 3,
    'PhD': 4
}

def generate_asp_facts():
    facts = []
    
    for _, row in students_df.iterrows():
        student_id = f"s{row.name}"  
        facts.append(f"student({student_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({student_id}, {edu_level}).")
        
        subject = row['Onderwerp'].lower().replace(' ', '_')
        facts.append(f"expertise({student_id}, {subject}).")

        availability = ast.literal_eval(row['Beschikbaarheid'])
        for day in availability:
            day_clean = day.lower()
            facts.append(f"availability({student_id}, {day_clean}).")
    
    for _, row in mentors_df.iterrows():
        mentor_id = f"m{row.name}"  
        facts.append(f"mentor({mentor_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({mentor_id}, {edu_level}).")
        
        subjects = ast.literal_eval(row['Onderwerpen'])
        for subject in subjects:
            subject_clean = subject.lower().replace(' ', '_')
            facts.append(f"expertise({mentor_id}, {subject_clean}).")

        max_students = row['Max_Studenten']
        facts.append(f"max_students({mentor_id}, {max_students}).")

        availability = ast.literal_eval(row['Beschikbaarheid'])
        for day in availability:
            day_clean = day.lower()
            facts.append(f"availability({mentor_id}, {day_clean}).")
    
    return "\n".join(facts)

asp_facts = generate_asp_facts()

ctl = Control()
ctl.add("base", [], f"""
{asp_facts}

% Valid mentor-student-day combination
valid_mentor(S, M, Day) :-
    student(S),
    mentor(M), 
    expertise(M, Subj), 
    expertise(S, Subj), 
    education(M, EM), 
    education(S, ES), 
    EM > ES,
    availability(S, Day),
    availability(M, Day).

% ONLY allow valid matches no invalid matches allowed (constraint)
:- match_mentor(S, M, Day), not valid_mentor(S, M, Day).

% Allow 0 or 1 mentor per student total
{{ match_mentor(S, M, Day) : valid_mentor(S, M, Day) }} <= 1 :- student(S).

% Each student can only be matched on one day (no multiple days)
:- match_mentor(S, M1, Day1), match_mentor(S, M2, Day2), Day1 != Day2.

% Each person can only have one match per day
:- match_mentor(S, M1, Day), match_mentor(S, M2, Day), M1 != M2.
:- match_mentor(S1, M, Day), match_mentor(S2, M, Day), S1 != S2.

% Allow max max_students per mentors (across all days)
:- mentor(M), max_students(M, Max), #count{{ S : match_mentor(S, M, Day) }} > Max.

% Maximize the number of matched students 
#maximize {{ 1,S : match_mentor(S,M,Day) }}.

% Output
#show match_mentor/3.
""")

ctl.ground([("base", [])])

def make_matches():
    matches = []
    
    def collect_matches(model):
        nonlocal matches
        matches = []
        
        for symbol in model.symbols(shown=True):
            if symbol.name == "match_mentor":
                student_id = symbol.arguments[0].name
                mentor_id = symbol.arguments[1].name
                day = symbol.arguments[2].name
                
                student_idx = int(student_id[1:])  
                mentor_idx = int(mentor_id[1:])    
                
                student_data = {
                    'voornaam': students_df.iloc[student_idx]['Voornaam'],
                    'achternaam': students_df.iloc[student_idx]['Achternaam'],
                    'opleidingsniveau': students_df.iloc[student_idx]['Opleidingsniveau'],
                    'onderwerp': students_df.iloc[student_idx]['Onderwerp'],
                    'beschikbaarheid': ast.literal_eval(students_df.iloc[student_idx]['Beschikbaarheid'])
                }
                
                mentor_data = {
                    'voornaam': mentors_df.iloc[mentor_idx]['Voornaam'],
                    'achternaam': mentors_df.iloc[mentor_idx]['Achternaam'],
                    'opleidingsniveau': mentors_df.iloc[mentor_idx]['Opleidingsniveau'],
                    'onderwerpen': ast.literal_eval(mentors_df.iloc[mentor_idx]['Onderwerpen']),
                    'max_studenten': mentors_df.iloc[mentor_idx]['Max_Studenten'],
                    'beschikbaarheid': ast.literal_eval(mentors_df.iloc[mentor_idx]['Beschikbaarheid'])
                }
                
                matches.append((student_data, mentor_data, day))
    
    result = ctl.solve(on_model=collect_matches)
    
    if result.satisfiable:
        return matches
    else:
        return []

# Usage
matches = make_matches()
print(f"Total matches: {len(matches)}")
for student, mentor, day in matches[:5]:
    print(f"{student['voornaam']} {student['achternaam']} -> {mentor['voornaam']} {mentor['achternaam']} on {day.capitalize()}")

def export_matches_to_csv(matches, filename='../DATASETS/matches.csv'):
    """Export matches to a CSV file with essential information"""
    if not matches:
        print("No matches to export")
        return
    
    # Prepare data for CSV
    rows = []
    for student, mentor, day in matches:
        row = {
            'Student': f"{student['voornaam']} {student['achternaam']}",
            'Mentor': f"{mentor['voornaam']} {mentor['achternaam']}",
            'Day': day.capitalize()
        }
        rows.append(row)
    
    # Create DataFrame and save to CSV
    matches_df = pd.DataFrame(rows)
    matches_df.to_csv(filename, index=False)
    print(f"Exported {len(matches)} matches to {filename}")
    
    return matches_df

# Export the matches
matches_df = export_matches_to_csv(matches)

print("\nPreview of exported matches:")
print(matches_df.head())

Total matches: 100
Amin de Lange -> Linde Schotte on Zondag
Sil Zaal -> Linde Schotte on Woensdag
Seth Molegraaf -> Linde Schotte on Zaterdag
Niek Krabbe -> Joëlle Bresé on Donderdag
Mika Van Bragt -> Joëlle Bresé on Zaterdag
Exported 100 matches to ../DATASETS/matches.csv

Preview of exported matches:
          Student         Mentor        Day
0   Amin de Lange  Linde Schotte     Zondag
1        Sil Zaal  Linde Schotte   Woensdag
2  Seth Molegraaf  Linde Schotte   Zaterdag
3     Niek Krabbe   Joëlle Bresé  Donderdag
4  Mika Van Bragt   Joëlle Bresé   Zaterdag


In [4]:
def debug_matching():
    matches = make_matches()
    print(f"Total matches found: {len(matches)}")
    
    if matches:
        print("\nSample matches:")
        for student, mentor, day in matches[:5]:
            print(f"  {student['voornaam']} {student['achternaam']} ({student['opleidingsniveau']}, {student['onderwerp']})")
            print(f"  -> {mentor['voornaam']} {mentor['achternaam']} ({mentor['opleidingsniveau']}, {mentor['onderwerpen']})")
            print(f"  Day: {day.capitalize()}\n")
    
    # Validation checks
    print("\n=== Validation Results ===")
    
    invalid_matches = get_invalid_matches(matches)
    if invalid_matches:
        print(f"[FAIL] Invalid matches: {len(invalid_matches)}")
        for student, mentor, day in invalid_matches[:3]:
            print(f"   - {student['voornaam']} {student['achternaam']} -> {mentor['voornaam']} {mentor['achternaam']} on {day}")
    else:
        print("[PASS] All matches are valid")
    
    over_capacity = check_mentor_capacity(matches)
    if over_capacity:
        print(f"[FAIL] Mentors over capacity:")
        for item in over_capacity:
            print(f"   - {item['mentor']}: {item['matched']} matched (max: {item['max']})")
    else:
        print("[PASS] All mentors within capacity")
    
    day_conflicts = check_day_conflicts(matches)
    if day_conflicts:
        print(f"[FAIL] Day conflicts found:")
        for conflict in day_conflicts:
            print(f"   - {conflict['type'].capitalize()} {conflict['name']} has {conflict['count']} matches on {conflict['day']}")
    else:
        print("[PASS] No day conflicts")
    
    multiple_days = check_multiple_days(matches)
    if multiple_days:
        print(f"[FAIL] Students matched on multiple days:")
        for item in multiple_days:
            print(f"   - {item['student']}: {', '.join(item['days'])}")
    else:
        print("[PASS] No students matched on multiple days")

debug_matching()

Total matches found: 100

Sample matches:
  Amin de Lange (Associate, Business Process Analytics)
  -> Linde Schotte (Bachelor, ['Software Architecture', 'Artificial Intelligence', 'Business Process Analytics', 'Business Change and Innovation'])
  Day: Zondag

  Sil Zaal (Associate, Business Change and Innovation)
  -> Linde Schotte (Bachelor, ['Software Architecture', 'Artificial Intelligence', 'Business Process Analytics', 'Business Change and Innovation'])
  Day: Woensdag

  Seth Molegraaf (Associate, Business Change and Innovation)
  -> Linde Schotte (Bachelor, ['Software Architecture', 'Artificial Intelligence', 'Business Process Analytics', 'Business Change and Innovation'])
  Day: Zaterdag

  Niek Krabbe (Associate, Data Science)
  -> Joëlle Bresé (Bachelor, ['Creative Digital Innovation', 'Data Science', 'Artificial Intelligence', 'Ethical Hacking'])
  Day: Donderdag

  Mika Van Bragt (Associate, Data Science)
  -> Joëlle Bresé (Bachelor, ['Creative Digital Innovation', 'Data S